In [2]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pandas_profiling import ProfileReport

from matplotlib import cm 

In [29]:
#load original lendingclub dataset
loan_data = pd.read_csv('accepted_2007_to_2018Q4.csv')

In [63]:
loan_data.shape
df = loan_data.copy()
df.shape

(2260701, 151)

In [33]:
#drop duplicate records
df.drop_duplicates(inplace = True)
df.shape

(2260701, 151)

In [64]:
# The model is to predict the loan defaults from the loans with at least 24 monthes long, so 2018 data were excluded from the analysis.
df = df.loc[:,:][df['issue_d'].str[-4:] != '2018']
df.shape

(1765459, 151)

In [69]:
df['issue_y'] = df['issue_d'].str[-4:]
df['issue_y'].value_counts()

2017    443579
2016    434407
2015    421095
2014    235629
2013    134814
2012     53367
2011     21721
2010     12537
2009      5281
2008      2393
2007       603
Name: issue_y, dtype: int64

In [42]:
df['loan_status'].value_counts()

Fully Paid                                             1029307
Current                                                 451136
Charged Off                                             259692
Late (31-120 days)                                       14246
In Grace Period                                           5517
Late (16-30 days)                                         2746
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     33
Name: loan_status, dtype: int64

In [70]:
#create outcome label 'good_bad' from feature 'loan_status'
df['good_bad'] = np.where(df['loan_status'].isin(['Charged Off', 'Default','Does not meet the credit policy. Status:Charged Off','Late (31-120 days)']), 1, 0) 
df['good_bad'].value_counts()

0    1490727
1     274732
Name: good_bad, dtype: int64

In [49]:
# table for bad_rate broken down by year

def bad_rate_by_cat(df, groupvar, dependcar):
    bad = df.groupby(groupvar)[dependcar].sum()
    count = df.groupby(groupvar)[dependcar].count()
    rate = df.groupby(groupvar)[dependcar].sum()/df.groupby(groupvar)[dependcar].count()
    bad_rate_by_cat = pd.concat((bad, count,rate), axis =1)
    bad_rate_by_cat.columns =  ['bad', 'total_count', 'bad_rate'] 
    
    return bad_rate_by_cat

bad_rate_by_cat(df, 'issue_y', 'good_bad')

,bad,total_count,bad_rate
issue_y,,,
2007,158,603,0.262023
2008,496,2393,0.207271
2009,723,5281,0.136906
2010,1757,12537,0.140145
2011,3297,21721,0.151789
2012,8644,53367,0.161973
2013,21027,134814,0.155970
2014,41496,235629,0.176107
2015,77163,421095,0.183244


In [71]:
# percentage of missing values for each feature
null_var =( df.isnull().sum()/len(df)).round(4).sort_values(ascending=False)
null_var

member_id                                     1.0000
orig_projected_additional_accrued_interest    0.9953
hardship_amount                               0.9940
hardship_type                                 0.9940
hardship_reason                               0.9940
hardship_status                               0.9940
deferral_term                                 0.9940
hardship_start_date                           0.9940
hardship_end_date                             0.9940
payment_plan_start_date                       0.9940
hardship_dpd                                  0.9940
hardship_loan_status                          0.9940
hardship_length                               0.9940
hardship_payoff_balance_amount                0.9940
hardship_last_payment_amount                  0.9940
sec_app_mths_since_last_major_derog           0.9923
settlement_date                               0.9809
settlement_term                               0.9809
settlement_percentage                         

In [72]:
# features with more than 50% missing values
feature_del_lst = (null_var[null_var > 0.5]).index.to_list()
len(feature_del_lst)

45

In [ ]:
# delete features with more than 50% missing values
df.drop(feature_del_lst, axis=1 , inplace = True)

In [76]:
df.shape

(1765459, 108)

In [94]:
# delete input features containing characteristic resulting from defaulted loans 
remove_lst = ['loan_status',
 'num_tl_120dpd_2m',
 'collection_recovery_fee',
 'last_pymnt_d',
 'collections_12_mths_ex_med',
 'delinq_amnt',
 'emp_title',
 'recoveries',
 'hardship_flag',
 'title',
 'chargeoff_within_12_mths',
 'num_tl_30dpd',
 'acc_now_delinq',
 'num_tl_90g_dpd_24m',
 'issue_y',
 'debt_settlement_flag',
 'tot_coll_amt']

df.drop(remove_lst, axis=1 , inplace = True)
df.shape

(1765459, 91)

In [97]:
# update: percentage of missing value for each feature
null_var1 =(( df.isnull().sum()/len(df)).round(4).sort_values(ascending=False))
feature_missing_lst = (null_var1[null_var1 > 0])
# len(feature_missing_lst)  total 49 features having missing value
feature_missing_lst

mths_since_last_delinq        0.4995
all_util                      0.4907
inq_fi                        0.4906
total_bal_il                  0.4906
inq_last_12m                  0.4906
total_cu_tl                   0.4906
max_bal_bc                    0.4906
open_rv_12m                   0.4906
open_rv_24m                   0.4906
open_il_24m                   0.4906
open_il_12m                   0.4906
open_act_il                   0.4906
open_acc_6m                   0.4906
mths_since_recent_inq         0.1326
mo_sin_old_il_acct            0.0684
emp_length                    0.0594
pct_tl_nvr_dlq                0.0399
mo_sin_rcnt_tl                0.0398
total_rev_hi_lim              0.0398
avg_cur_bal                   0.0398
mo_sin_old_rev_tl_op          0.0398
mo_sin_rcnt_rev_tl_op         0.0398
tot_cur_bal                   0.0398
num_tl_op_past_12m            0.0398
tot_hi_cred_lim               0.0398
num_rev_tl_bal_gt_0           0.0398
num_rev_accts                 0.0398
n